In [5]:
#!pip install statsmodels 
!pip install xgboost

  Using cached xgboost-1.5.2-py3-none-win_amd64.whl (106.6 MB)


In [29]:
#loading packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

%matplotlib inline
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.preprocessing import LabelEncoder
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [30]:
#load data
df = pd.read_csv('data/flights_model.csv')
df.head()

,crs_elapsed_time,air_time,distance,carrier,mean_origin,mean_dest,mean_hist_dep_delay,mean_hist_arr_delay,crs_dep_time_1,crs_dep_time_2,...,dest_363,dest_364,dest_365,dest_366,dest_367,dest_368,dest_369,dest_370,dest_371,arr_delay
0,-0.330585,-0.423369,-0.502097,-0.587808,-0.827059,0.897653,0.640120,0.411227,-0.028143,-0.015226,...,-0.011153,-0.031468,-0.009209,-0.005824,-0.04277,-0.005317,-0.009511,-0.01366,-0.014464,-14.0
1,0.757762,0.915106,0.556956,-0.934657,1.363291,1.333490,0.889199,0.963773,-0.028143,-0.015226,...,-0.011153,-0.031468,-0.009209,-0.005824,-0.04277,-0.005317,-0.009511,-0.01366,-0.014464,19.0
2,0.325214,0.368202,0.440989,-0.587808,0.257010,-0.186138,1.252072,1.073679,-0.028143,-0.015226,...,-0.011153,-0.031468,-0.009209,-0.005824,-0.04277,-0.005317,-0.009511,-0.01366,-0.014464,-14.0
3,-0.121287,-0.092348,-0.056988,-0.934657,1.046254,1.112326,1.089943,1.073679,-0.028143,-0.015226,...,-0.011153,-0.031468,-0.009209,-0.005824,-0.04277,-0.005317,-0.009511,-0.01366,-0.014464,-14.0
4,-0.358491,-0.365800,-0.239465,1.493283,0.503874,-0.203897,0.889199,1.086117,-0.028143,-0.015226,...,-0.011153,-0.031468,-0.009209,-0.005824,-0.04277,-0.005317,-0.009511,-0.01366,-0.014464,-17.0


In [32]:
col=['air_time','carrier']
df.drop(col,axis=1,inplace=True)

# Setting x and y values

In [11]:
X = df.drop('arr_delay',axis=1)
y = df['arr_delay']

# Lasso = Feature Importance

In [4]:
reg = LassoCV()
reg.fit(X, y)
coef = pd.Series(reg.coef_, index = X.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

import matplotlib.pyplot as plt
import matplotlib
imp_coef = coef.sort_values()

#matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
#imp_coef.plot(kind = "barh")
#plt.title("Feature importance using Lasso Model")

Lasso picked 672 variables and eliminated the other 154 variables


# Splitting of data

In [12]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Lasso Regression

In [6]:
reg = LassoCV()
reg.fit(X, y)
model_lasso = Lasso(alpha=0.01)
model_lasso.fit(X_train, y_train) 
pred_test_lasso= model_lasso.predict(X_test)
print('RSME: ',np.sqrt(mean_squared_error(y_test,pred_test_lasso))) 
print('R2: ',r2_score(y_test, pred_test_lasso))

RSME:  13.309292138385318
R2:  0.21351092323699117


# Ridge Regression

In [23]:
rr = Ridge(alpha=0.01)
rr.fit(X_train, y_train) 
pred_test_rr= rr.predict(X_test)
print('RSME: ',np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print('R2: ',r2_score(y_test, pred_test_rr))

RSME:  13.378960647641172
R2:  0.2108460121457455


# Random Forest

In [13]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train,y_train)
y_pred_RF = rf.predict(X_test)
from sklearn import metrics
print('R2 score:',r2_score(y_test, y_pred_RF))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_RF)))

R2 score: 0.16152954235697714
RMSE: 13.798270400183366


# XGBoost

In [14]:
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 15, alpha = 10, n_estimators = 200)
xg_reg.fit(X_train, y_train)
y_pred_xgb = xg_reg.predict(X_test)
print('Model RMSE is: ',np.sqrt(mean_squared_error(y_test, y_pred_xgb)))
print('Model R2 Score is: ',r2_score(y_test,y_pred_xgb))

C:\Users\My account\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Model RMSE is:  13.830704799285373
Model R2 Score is:  0.15758306972384373


## Predict on January 2020 Flights_test dataset
Files can be found in:
data/test_eval.csv

In [18]:
## Import scaled test dataset (see: 5_feature_selection_test)
test_eval = pd.read_csv('data/flights_model.csv')

In [19]:
test_eval.head()
test_eval = test_eval.drop('arr_delay',axis=1)

In [20]:
pred_test = xg_reg.predict(test_eval)

C:\Users\My account\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [21]:
pd.DataFrame(pred_test).to_csv('data/pred_test.csv')

## Evaluate on the test data and prepare submission file

In [22]:
df = pd.read_csv("data/flights_test.csv")
df_submission = df.copy()
df_submission = df_submission[["fl_date","mkt_unique_carrier","mkt_carrier_fl_num", "origin","dest"]]
df_submission.sample(5)

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest
138036,2020-01-07,AA,5414,STL,DCA
116684,2020-01-06,AA,2590,SNA,ORD
114267,2020-01-06,DL,6065,BOS,LGA
68326,2020-01-04,UA,1896,ORD,SJU
136391,2020-01-07,DL,1985,SEA,SLC


In [23]:
y = pd.read_csv('data/pred_test.csv')

In [24]:
y_pred = y["0"].values

In [25]:
y_predicted = pd.DataFrame(data = y_pred,
                    columns = ["predicted_delay"])

In [26]:
final_pred = df_submission.merge(y_predicted, right_index = True, left_index = True)
final_pred.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,predicted_delay
0,2020-01-01,WN,5888,ONT,SFO,-12.295573
1,2020-01-01,WN,6276,ONT,SFO,8.637155
2,2020-01-01,WN,4598,ONT,SJC,-6.269744
3,2020-01-01,WN,4761,ONT,SJC,-3.930248
4,2020-01-01,WN,5162,ONT,SJC,-8.441968


In [27]:
final_pred.to_csv("submission.csv", index=False)

In [28]:
final_pred

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,predicted_delay
0,2020-01-01,WN,5888,ONT,SFO,-12.295573
1,2020-01-01,WN,6276,ONT,SFO,8.637155
2,2020-01-01,WN,4598,ONT,SJC,-6.269744
3,2020-01-01,WN,4761,ONT,SJC,-3.930248
4,2020-01-01,WN,5162,ONT,SJC,-8.441968
...,...,...,...,...,...,...
150618,2020-01-07,DL,4813,DTW,JFK,-0.722606
150619,2020-01-07,DL,4814,GSP,LGA,-4.348841
150620,2020-01-07,DL,4815,ATL,XNA,-20.867384
150621,2020-01-07,DL,4815,XNA,ATL,-6.872995
